In [2]:
!pip install pandas


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------- ------------------ 6.3/11.5 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    --------------------------------------- 0.3/12.6 MB ? eta -:--:--
   --------------------- ------------------ 6.8/12.6 MB 38.1 MB/s eta 0:00:01
   ----------------------------------- ---- 11.3/12.6 MB 29.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 24.8 MB/s eta 0:00:00


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head(1)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0


In [35]:
df = df[["text", "label_num"]]
df.head()
df.columns

Index(['text', 'label_num'], dtype='object')

In [6]:
df.iloc[0, 0]

"Subject: enron methanol ; meter # : 988291\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\nflow data provided by daren } .\nplease override pop ' s daily volume { presently zero } to reflect daily\nactivity you can obtain from gas control .\nthis change is needed asap for economics purposes ."

In [8]:
# natural-native language tool kit
!pip install nltk

In [13]:
import nltk
nltk.download("stopwords") # kütüphanenin arkasındaki veri setidir bu defaultta yüklü gelmez sen işine yarayacak olanı indirirsin

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Barış\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [21]:
# Python, PYTHON, PYtHoN, PYThon
# Pytgon. Python, Python:
import string

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
text = df.iloc[0, 0]

text = text.lower()

text = text.translate(str.maketrans('','',string.punctuation)) #bütün noktalama işaretlerini içeren bir değişken punctuation

words = text.split() # splite parametre yollamaz isen her boşluk ile bir array elementi oluşturur

words = [word for word in words if not word.isdigit()] # bu array wordsdeki her elementi gezer ve numeric olmayanları alır

# Stem, Remove Stopwords->nltk kütüphanesi ile yapılır 
# Stem -> kelimeleri köküne indirerek aynı köke sahio kelimelerin saklanmasını sağlar (Koşuyor->koş)
# Remove Stopwords -> The,at,is 

words = [ ps.stem(word) for word in words if word not in stopwords.words("english")] # bu kelimeleri gezer ve ingilizcedeki propositionların hepsini kaldırır
                                                                           # propositions-> this,is,a,at,an,in gibi
                                                                           #baştaki stem ile de kelimeleri köküne indirdik
" ".join(words) # join ise ilk verdiğin karaktere göre verilen arrayi birleştiriyor yan yana

'subject enron methanol meter follow note gave monday preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [24]:
def clean_text(text):
    text = text.lower()

    text = text.translate(str.maketrans('','',string.punctuation)) 

    words = text.split() 

    words = [word for word in words if not word.isdigit()] 

    words = [ ps.stem(word) for word in words if word not in stopwords.words("english")] 

    return " ".join(words)

In [25]:
clean_text("Merhaba, bu e-posta 45. python dersi hakkındadır.")

'merhaba bu eposta python dersi hakkındadır'

In [27]:
df['clean_text'] = df['text'].apply(clean_text)

In [28]:
df.head()

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\nth...,0,subject enron methanol meter follow note gave ...
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0,subject hpl nom januari see attach file hplnol...
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0,subject neon retreat ho ho ho around wonder ti...
3,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window offic cheap main tren...
4,Subject: re : indian springs\nthis deal is to ...,0,subject indian spring deal book teco pvr reven...


In [32]:
X = df['clean_text']
y = df['label_num']
!pip install scikit-learn

# Vektorize -> TfIDF Vectorizer  metinleri numeric hale getiriyor bir nevi
from sklearn.feature_extraction.text import TfidfVectorizer
#ilk ödev bu hafta vectorize nedir tfıdf vectorizer nasıl çalışır araştır


In [60]:
tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(X)
X_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 307420 stored elements and shape (5171, 37932)>

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

In [35]:
# Naive Bayes (bayes algoritması)

In [21]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [22]:
y_pred = model.predict(X_test)

In [23]:
y_pred

array([0, 1, 0, ..., 0, 0, 0], shape=(1552,))

In [24]:
from sklearn.metrics import classification_report

In [25]:
print(classification_report(y_test, y_pred))
#support -> veri sayısı
# precisions -> kesinlik  -> söylenilen ne kadar doğru -> kısaca spam olmayan bazı maillere spam denmiş 
#0 olanlar spam olmayan 1 olanlar spam

# recall-> duyarlılık -> ne kadarını yakalamışım
# f1-score ->precision-recallin oranı gibi

              precision    recall  f1-score   support

           0       0.89      1.00      0.94      1121
           1       1.00      0.68      0.81       431

    accuracy                           0.91      1552
   macro avg       0.95      0.84      0.88      1552
weighted avg       0.92      0.91      0.91      1552



In [26]:
#bu modeli siz geliştiriyor olsaydnız, hangi değişiklikleri ile modelin performansını arttırmak için ne uygularsınız gösterin
#ilk ödev bu hafta vectorize nedir tfıdf vectorizer nasıl çalışır araştır
#haftaya KNN ALGORİTMASI ANLATILACAK

In [61]:
kelimeler = tfidf.get_feature_names_out()

In [62]:
log_probs_spam = model.feature_log_prob_[1]
log_probs_not_spam = model.feature_log_prob_[0]

In [68]:
xd = pd.DataFrame({
    "Kelimeler": kelimeler,
    "spam_LogProb": log_probs_spam,
    "Notspam_logprobs": log_probs_not_spam,
    "fark": log_probs_spam - log_probs_not_spam
})
xd = xd.sort_values(by="fark", ascending=False)
xd.head(5)

,Kelimeler,spam_LogProb,Notspam_logprobs,fark
25725,pill,-8.094734,-10.834261,2.739527
26487,prescript,-8.094332,-10.814706,2.720374
21381,med,-8.274477,-10.834261,2.559784
6360,ciali,-8.375865,-10.834261,2.458396
35278,viagra,-8.394065,-10.834261,2.440196


Doğru uygulayabildiysem burdaki fark stünumuz kelimelerin spam olma ihtimallerini gösteriyor bu değer ne kadar yüksekse kelimeler o kadar spam olarak ölçülmüş

direkt coef tablosu uygulamak istedim fakat hatalar aldım çözemedim gpt yardımıyla log yöntemine baktım kısaca 'pill' kelimesinin geçtiği cümleler daha çok spam olarak tahmin edilmiş

benim fikrime göre spam olmayan fakat spam olarak tahmin edilen cümlelerdeki kelimelerin burdaki log tablosu ile karşılaştırılıp bu cümlelerdeki kelimelerden yola çıkarak biraz daha kelime temizliği yaparsak modelin verimliliğini arttırabiliriz